# IST 691: Deep Learning in Practice

**Homework 3**

Name: Zane Alderfer

SUID: 503765874

*Save this notebook into your Google Drive. The notebook has appropriate comments at the top of code cells to indicate whether you need to modify them or not. Answer your questions directly in the notebook. Remember to use the GPU as your runtime. Once finished, run ensure all code blocks are run, download the notebook and submit through Blackboard.*

### Setup

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import string
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# to build nearest neighbor model
from sklearn.neighbors import NearestNeighbors

2024-05-28 15:39:10.043150: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or 

In [9]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=6a0f45fd71f51d323dc2bf6f08da83b16e778bcd1daa6d985083684f020f3cf8
  Stored in directory: /Users/zanealderfer/Library/Caches/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In this homework, we will perform **sarcasm detection** with [Onion](https://www.theonion.com/) vs [HuffPost](https://www.huffpost.com/) headlines, using LSTM. We will first load the data and generate the training and testing input and labels.

In [12]:
!wget -nc -q https://github.com/mrech/NLP_TensorFlow/blob/master/0_Sentiment_in_Text/Sarcasm_Headlines_Dataset_v2.json?raw=true

/bin/bash: wget: command not found


In [21]:
# Path to the JSON file
file_path = '/Users/zanealderfer/Downloads/Sarcasm_Headlines_Dataset_v2.json'

# Read the JSON file into a DataFrame
try:
    df = pd.read_json(file_path, lines=True)  # Use lines=True if each line is a valid JSON object
    print(df.head())  # Display the first few rows of the DataFrame
except ValueError as e:
    print(f"Error: {e}")

   is_sarcastic                                           headline  \
0             1  thirtysomething scientists unveil doomsday clo...   
1             0  dem rep. totally nails why congress is falling...   
2             0  eat your veggies: 9 deliciously different recipes   
3             1  inclement weather prevents liar from getting t...   
4             1  mother comes pretty close to using word 'strea...   

                                        article_link  
0  https://www.theonion.com/thirtysomething-scien...  
1  https://www.huffingtonpost.com/entry/donna-edw...  
2  https://www.huffingtonpost.com/entry/eat-your-...  
3  https://local.theonion.com/inclement-weather-p...  
4  https://www.theonion.com/mother-comes-pretty-c...  


In [22]:
# get information about the data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [23]:
# take a peek at the key data
df[['headline', 'is_sarcastic']].head(5).values

array([['thirtysomething scientists unveil doomsday clock of hair loss',
        1],
       ['dem rep. totally nails why congress is falling short on gender, racial equality',
        0],
       ['eat your veggies: 9 deliciously different recipes', 0],
       ['inclement weather prevents liar from getting to work', 1],
       ["mother comes pretty close to using word 'streaming' correctly",
        1]], dtype=object)

In [24]:
# the training input sequence will be in variable seq_padd_train and the label in train_y
# The testing input sequence will be in variable seq_padd_test and the label in test_y
headlines = df['headline'].values.tolist()
sarcastic = df['is_sarcastic'].values.tolist()

In [25]:
training_size = 20000
test_size = 6709

train_x = headlines[:training_size]
test_x = headlines[training_size:]
train_y = np.array(sarcastic[:training_size])
test_y = np.array(sarcastic[training_size:])

# sequence of words input
max_len = 16

tokenizer = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(train_x)

word_index = tokenizer.word_index
index_word = {v: k for k, v in word_index.items()}
vocab_size = len(word_index)
sequence_train = tokenizer.texts_to_sequences(train_x)
seq_padd_train = pad_sequences(sequence_train,
                               padding = 'post',
                               truncating = 'post',
                               maxlen = max_len)


sequence_test = tokenizer.texts_to_sequences(test_x)
seq_padd_test = pad_sequences(sequence_test, padding = 'post',
                              truncating = 'post',
                              maxlen = max_len)

### Q1 Calculating the Trainable Parameters of an LSTM

Below is the summary of an LSTM neural network with embeddings and three layers. Explain in detail, after this cell, the "why" of the number of parameters of each of the layers displayed by `model1.summary()`. Cite any sources you used to answer this question.

`model1.summary()`
```
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000100   
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 96)          86400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                41216     
_________________________________________________________________
predictions (Dense)          (None, 1)                 65        
=================================================================
Total params: 2,245,029
Trainable params: 2,245,029
Non-trainable params: 0
_________________________________________________________________
```

**Why do we have the number of parameters after each of the layers?**

The InputLayer does not have any parameters. It simply defines the shape of the input data, which in this case is a sequence of variable length.

The embedding layer maps input integers to dense vectors of fixed size. The number of parameters is calculated as:
Number of parameters = Vocabulary size × Embedding dimension

An LSTM layer has the following parameters:
Parameters = 4 × (Input dimension × Units + Units^2 + Units)

A dense layer has the following parameters:
Parameters = Input dimension × Units + Units

The total number of parameters is the sum of parameters from all layers:

Total parameters = 2,000,100+ 117,248 + 86,400 + 41,216 + 65 = 2,245,029
 
References:

Brownlee, J. (2017). How to Configure the Number of Layers and Nodes in an LSTM Network. Machine Learning Mastery.
Understanding LSTM Networks. (2015). Retrieved from colah's blog.

### Q2: LSTM for Detecting Sarcasm

Modify the code below to create an embedding layer of dimension 50. The vocabulary size is in variable `vocab_size`, and remember to add one in the embedding for the "out of vocabulary" input. Define an LSTM with two layers, one with 64 memory size and the second with 32 memory size. Remember to use the suffix `2` for each of the variables you define (e.g., `x2`)

In [27]:
# an integer input for vocab indices
inputs2 = tf.keras.Input(shape = (None,), dtype = 'int32')

embedding_dim = 50

# define the layers below Embedding -> LSTM 1 -> LSTM 2
x2 = layers.Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, name='embedding2')(inputs2)
x2 = layers.LSTM(64, return_sequences=True, name='lstm1_2')(x2)
x2 = layers.LSTM(32, name='lstm2_2')(x2)

# we project onto a single unit output layer, and squash it with a sigmoid
predictions2 = layers.Dense(1, activation = 'sigmoid', name = 'predictions')(x2)

model2 = tf.keras.Model(inputs2, predictions2, name = 'lstm_simple')

# compile the model with binary crossentropy loss and an adam optimizer
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [28]:
epochs = 10
# fit the model using the train and test datasets
model2.fit(seq_padd_train, train_y,
           validation_split = 0.1,
           epochs = epochs,
           verbose = 2,
           batch_size = 64)

Epoch 1/10
282/282 - 18s - loss: 0.4286 - accuracy: 0.7887 - val_loss: 0.3297 - val_accuracy: 0.8555 - 18s/epoch - 62ms/step
Epoch 2/10
282/282 - 11s - loss: 0.1815 - accuracy: 0.9311 - val_loss: 0.3735 - val_accuracy: 0.8355 - 11s/epoch - 39ms/step
Epoch 3/10
282/282 - 10s - loss: 0.0838 - accuracy: 0.9727 - val_loss: 0.4960 - val_accuracy: 0.8300 - 10s/epoch - 37ms/step
Epoch 4/10
282/282 - 11s - loss: 0.0509 - accuracy: 0.9838 - val_loss: 0.5862 - val_accuracy: 0.8370 - 11s/epoch - 39ms/step
Epoch 5/10
282/282 - 22s - loss: 0.0306 - accuracy: 0.9906 - val_loss: 0.6400 - val_accuracy: 0.8310 - 22s/epoch - 78ms/step
Epoch 6/10
282/282 - 20s - loss: 0.0179 - accuracy: 0.9947 - val_loss: 0.8234 - val_accuracy: 0.8260 - 20s/epoch - 69ms/step
Epoch 7/10
282/282 - 16s - loss: 0.0140 - accuracy: 0.9957 - val_loss: 0.8475 - val_accuracy: 0.8315 - 16s/epoch - 56ms/step
Epoch 8/10
282/282 - 12s - loss: 0.0110 - accuracy: 0.9974 - val_loss: 0.8335 - val_accuracy: 0.8295 - 12s/epoch - 43ms/step


In [29]:
# estimate the test performance
model2.evaluate(seq_padd_test, test_y)

270/270 [==============================] - 5s 17ms/step - loss: 0.8624 - accuracy: 0.8286


[0.8624102473258972, 0.8286344408988953]

### Q3: GloVe Word Embeddings

Use the code below to download the GloVe embeddings and create the matrix `embedding_matrix` corresponding to the vocabulary above. Define a layer `embedding_layer_glove` which will be use by the LSTM below. Evaluate the performance and compare to model above.

In [30]:
import os
embeddings_index = {}
f = open('/Users/zanealderfer/Downloads/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype = 'float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [31]:
num_tokens = vocab_size + 2
embedding_dim3 = 100
hits = 0
misses = 0

# prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim3))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        # this includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 21242 words (4656 misses)


Create the embedding layer below:

In [32]:
# create the embedding layer using the embedding_matrix from above
embedding_layer_glove = layers.Embedding(
    num_tokens,
    embedding_dim3,
    input_length = max_len,
    embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix),
    trainable = False,
)

In [33]:
# an integer input for vocab indices
inputs3 = tf.keras.Input(shape = (None,), dtype = 'int32')

# next, we add a layer to map those vocab indices into a space of dimensionality
x3 = embedding_layer_glove(inputs3)

x3 = layers.LSTM(32)(x3)

# we project onto a single unit output layer, and squash it with a sigmoid
predictions3 = layers.Dense(1, activation = 'sigmoid', name = 'predictions')(x3)

model3 = tf.keras.Model(inputs3, predictions3)

# compile the model with binary crossentropy loss and an adam optimizer.
model3.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [34]:
# fit the model using the train and test datasets
epochs = 10
model3.fit(seq_padd_train, train_y,
           validation_split = 0.1,
           epochs = epochs,
           verbose = 2,
           batch_size = 64)

Epoch 1/10
282/282 - 6s - loss: 0.5367 - accuracy: 0.7200 - val_loss: 0.4460 - val_accuracy: 0.7990 - 6s/epoch - 20ms/step
Epoch 2/10
282/282 - 2s - loss: 0.4082 - accuracy: 0.8181 - val_loss: 0.3969 - val_accuracy: 0.8210 - 2s/epoch - 8ms/step
Epoch 3/10
282/282 - 2s - loss: 0.3665 - accuracy: 0.8382 - val_loss: 0.3670 - val_accuracy: 0.8370 - 2s/epoch - 8ms/step
Epoch 4/10
282/282 - 3s - loss: 0.3313 - accuracy: 0.8579 - val_loss: 0.3651 - val_accuracy: 0.8405 - 3s/epoch - 9ms/step
Epoch 5/10
282/282 - 2s - loss: 0.3059 - accuracy: 0.8699 - val_loss: 0.3415 - val_accuracy: 0.8545 - 2s/epoch - 8ms/step
Epoch 6/10
282/282 - 2s - loss: 0.2906 - accuracy: 0.8770 - val_loss: 0.3443 - val_accuracy: 0.8535 - 2s/epoch - 8ms/step
Epoch 7/10
282/282 - 2s - loss: 0.2692 - accuracy: 0.8886 - val_loss: 0.3419 - val_accuracy: 0.8525 - 2s/epoch - 8ms/step
Epoch 8/10
282/282 - 2s - loss: 0.2531 - accuracy: 0.8946 - val_loss: 0.3424 - val_accuracy: 0.8590 - 2s/epoch - 9ms/step
Epoch 9/10
282/282 - 3s

In [35]:
model3.evaluate(seq_padd_test, test_y)

270/270 [==============================] - 1s 5ms/step - loss: 0.3492 - accuracy: 0.8551


[0.3492344319820404, 0.8550875782966614]

Is it better or worse performance compared to `model2`? Why?

Model3 is worse compared to model2.  Fewer LSTM layers (as in model3) may be less prone to overfitting and faster to train but might not capture as complex patterns.  Model2 could potentially perform better on large datasets with sufficient training data because its trainable embeddings and multiple LSTM layers can learn more specific patterns and features from the data

### Q4: Word Analogies

Above, we created the matrix `embedding_matrix` for the vocabulary in the sarcasm dataset. Use the code below to find the word analogy to "`germany` is to `berlin` as `uk` is to _blank_"

In [36]:
# we will first create the nearest neighbor model
nbrs_glove = NearestNeighbors(n_neighbors = 5, metric = 'cosine').fit(embedding_matrix)

In [37]:
# let's check if it works
embedding_man = embedding_matrix[word_index['man']]

In [38]:
# closest words to `man`
dist, idx = nbrs_glove.kneighbors([embedding_man])
[index_word[i] for i in idx[0]]

['man', 'woman', 'boy', 'one', 'person']

In [39]:
# now define the proper embedding to solve the analogy
germany_embedding = embedding_matrix[word_index['germany']]
berlin_embedding = embedding_matrix[word_index['berlin']]
uk_embedding = embedding_matrix[word_index['uk']]
blank_embedding = berlin_embedding - germany_embedding + uk_embedding

In [40]:
# find the closest to blank_embedding
# closest words to `man`
dist, idx = nbrs_glove.kneighbors([blank_embedding])
[index_word[i] for i in idx[0]]

['uk', 'london', 'theatre', '2013', '2011']

### Q5: Biases

As we discussed in class, there might be several biases in word embeddings. Use the list of occupations below and for each of them find whether `man` or `woman` is closest to it. In particular, first list all occupations that are closer to `man` than `woman`, and then all occupations that are closer to `woman` than `man`.

_Hint_: Use the `cosine` distance between pairs of embeddings from the `SciPy` package. If the ocupation does not exist in the embedding matrix, skip it. Also, remember that the cosine distance is smaller when the embeddings are more similar.


In [41]:
from scipy.spatial.distance import cosine
print('cosine([1,1], [1,1]): ', cosine([1,1], [1,1]))
print('cosine([1,1], [0,1]): ', cosine([1,1], [0,1]))

cosine([1,1], [1,1]):  0
cosine([1,1], [0,1]):  0.29289321881345254


In [42]:
occupation_list = """technician, accountant, supervisor, engineer, worker, educator, clerk, counselor,
inspector, mechanic, manager, therapist, administrator, salesperson, receptionist, librarian,
advisor, pharmacist, janitor, psychologist, physician, carpenter, nurse, investigator,
bartender, specialist, electrician, officer, pathologist, teacher, lawyer, planner, practitioner,
plumber, instructor, surgeon, veterinarian, paramedic, examiner, chemist, machinist,
appraiser, nutritionist, architect, hairdresser, baker, programmer, paralegal, hygienist,
scientist""".replace('\n', '').replace(' ', '').split(',')

In [43]:
man_embedding = embedding_matrix[word_index['man']]
woman_embedding = embedding_matrix[word_index['woman']]

In [44]:
# Lists to hold occupations closer to 'man' and 'woman'
closer_to_man = []
closer_to_woman = []

# Calculate distances and categorize occupations
for occupation in occupation_list:
    if occupation in word_index:
        occupation_embedding = embedding_matrix[word_index[occupation]]
        dist_to_man = cosine(occupation_embedding, man_embedding)
        dist_to_woman = cosine(occupation_embedding, woman_embedding)
        
        if dist_to_man < dist_to_woman:
            closer_to_man.append(occupation)
        else:
            closer_to_woman.append(occupation)

# Print the results
print("Occupations closer to 'man':")
print(closer_to_man)
print("\nOccupations closer to 'woman':")
print(closer_to_woman)

Occupations closer to 'man':
['engineer', 'inspector', 'mechanic', 'manager', 'advisor', 'carpenter', 'investigator', 'officer', 'lawyer', 'planner', 'plumber', 'instructor', 'architect', 'scientist']

Occupations closer to 'woman':
['technician', 'supervisor', 'worker', 'educator', 'clerk', 'counselor', 'therapist', 'administrator', 'receptionist', 'librarian', 'pharmacist', 'janitor', 'psychologist', 'physician', 'nurse', 'bartender', 'teacher', 'practitioner', 'surgeon', 'veterinarian', 'paramedic', 'examiner', 'nutritionist', 'hairdresser', 'hygienist']


Do you see a pattern in the results? Do you think there are biases?

It seems like the men have a lot of technical or managerial jobs like engineer, carpenter, architect, and scientist while the women seems to have more nurturing or supportive roles like nurse, hairdresser, hygienist, clerk, educator.  There could be potential biases in historical roles as well as societal biases within the Glove embeddings.

### Q6: Sequence to Sequence Embedding

What is the problem with LSTM models, and why do we need **attention** to fix them? Give as an example of what happens with sequence to sequence models for translation.

LSTMs struggle with very long sequences, leading to loss of important information over time. The use of a single fixed-size context vector compresses the entire input sequence, which can be limiting for long and complex sequences. As the sequence length increases, important details may be lost. Additionally, LSTMs find it hard to align parts of the input sequence with the corresponding parts of the output sequence, which is crucial in tasks like translation. The attention mechanism addresses these issues by creating context vectors dynamically for each output step, focusing on different parts of the input sequence as needed. It preserves important information by concentrating on relevant parts of the input at each decoding step. The mechanism also learns which parts of the input sequence are relevant to each part of the output sequence, thus improving accuracy. Furthermore, attention weights can be visualized, showing which parts of the input the model focuses on for each output step, enhancing interpretability. In sequence-to-sequence models for translation, the standard seq2seq with LSTM without attention processes the input sequence to produce a single context vector during encoding. During decoding, it uses this fixed context vector, which may struggle to capture all nuances, especially for long sequences. With attention, the encoding phase produces a sequence of hidden states. The decoding phase generates a context vector for each output step by computing attention weights over the hidden states, dynamically focusing on different parts of the input. This allows the model to dynamically focus on different parts of the input sequence, leading to more accurate translations by effectively aligning input and output sequences.